#Before starting, I would like to give an overview of how to structure any deep learning project.

**Preprocess and load data**- we need to process it before feeding to the neural network. In this step, we will also visualize data which will help us to gain insight into the data.

**Define model**- Now we need a neural network model. This means we need to specify the number of hidden layers in the neural network and their size, the input and output size.

**Loss and optimizer**- Now we need to define the loss function according to our task. We also need to specify the optimizer to use with learning rate and other hyperparameters of the optimizer.

**Fit model**- This is the training step of the neural network. Here we need to define the number of epochs for which we need to train the neural network.

In [1]:
pip install pandas

In [1]:
 import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, concatenate

# Sample data (replace with your custom data)
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 4],
    'movie_id': [1, 2, 3, 1, 2, 1, 3, 2],
    'rating': [5, 4, 3, 4, 5, 2, 3, 4]
}
df = pd.DataFrame(data)

# Normalize the user and movie IDs
user_ids = df['user_id'].unique().tolist()
user_id_mapping = {id: i for i, id in enumerate(user_ids)}
movie_ids = df['movie_id'].unique().tolist()
movie_id_mapping = {id: i for i, id in enumerate(movie_ids)}

df['user_id'] = df['user_id'].map(user_id_mapping)
df['movie_id'] = df['movie_id'].map(movie_id_mapping)

# Split the data
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Define the model
n_users = df['user_id'].nunique()
n_movies = df['movie_id'].nunique()
n_latent_factors = 10

user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=n_users, output_dim=n_latent_factors, name='user_embedding')(user_input)
user_vec = Flatten(name='user_flatten')(user_embedding)

movie_input = Input(shape=(1,), name='movie_input')
movie_embedding = Embedding(input_dim=n_movies, output_dim=n_latent_factors, name='movie_embedding')(movie_input)
movie_vec = Flatten(name='movie_flatten')(movie_embedding)

concat = concatenate([user_vec, movie_vec], name='concat')
dense = Dense(128, activation='relu', name='dense')(concat)
output = Dense(1, name='output')(dense)

model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
train_user_data = train_data['user_id'].values
train_movie_data = train_data['movie_id'].values
train_ratings = train_data['rating'].values

model.fit([train_user_data, train_movie_data], train_ratings, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
test_user_data = test_data['user_id'].values
test_movie_data = test_data['movie_id'].values
test_ratings = test_data['rating'].values

loss = model.evaluate([test_user_data, test_movie_data], test_ratings)
print(f'Test Loss: {loss}')

# Make predictions
user_id = 1  # Example user
movie_id = 2  # Example movie

predicted_rating = model.predict([np.array([user_id_mapping[user_id]]), np.array([movie_id_mapping[movie_id]])])
print(f'Predicted rating for user {user_id} and movie {movie_id}: {predicted_rating[0][0]}')


Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 18.0182 - val_loss: 8.9121
Epoch 2/10
1/1 [==============================] - 0s 64ms/step - loss: 17.9051 - val_loss: 8.8573
Epoch 3/10
1/1 [==============================] - 0s 77ms/step - loss: 17.7916 - val_loss: 8.7989
Epoch 4/10
1/1 [==============================] - 0s 73ms/step - loss: 17.6790 - val_loss: 8.7402
Epoch 5/10
1/1 [==============================] - 0s 66ms/step - loss: 17.5662 - val_loss: 8.6812
Epoch 6/10
1/1 [==============================] - 0s 71ms/step - loss: 17.4532 - val_loss: 8.6228
Epoch 7/10
1/1 [==============================] - 0s 55ms/step - loss: 17.3401 - val_loss: 8.5636
Epoch 8/10
1/1 [==============================] - 0s 86ms/step - loss: 17.2245 - val_loss: 8.5017
Epoch 9/10
1/1 [==============================] - 0s 56ms/step - loss: 17.1050 - val_loss: 8.4376
Epoch 10/10
1/1 [==============================] - 0s 63ms/step - loss: 9.1844
Test Loss: 9.184415817260742
1/1 [=======